In [2]:
import inspect
import operator as op

from functools import partial, wraps
from typing import *

In [10]:
def identity(x: Any):
    return x    

In [11]:
def evaluable(dtype: Type[Any]) -> bool:
    return dtype == Any or hasattr(dtype, "__bool__") or hasattr(dtype, "__len__")


def invertible(dtype: Type[Any]) -> bool:
    return dtype == Any or hasattr(dtype, "__invert__")


class Curry:
    def __init__(self, func: Callable):
        self._func = func
        try:
            self._signature = inspect.signature(func)
        except ValueError:
            self._signature = inspect.signature(func.__call__)
        
        self.__doc__ = getattr(func, '__doc__', None)
        self.__name__ = getattr(func, '__name__', '<curry>')
        self.__module__ = getattr(func, '__module__', None)
        self.__qualname__ = getattr(func, '__qualname__', None)
    
    def __call__(self, *args, **kwargs):
        func = partial(self._func, *args, **kwargs)
        
        is_ready = True
        for param in self.signature.parameters.values():
            if param.kind in (param.POSITIONAL_ONLY, param.POSITIONAL_OR_KEYWORD) \
            and param.default == param.empty:
                is_ready = False
                
        if is_ready:
            return func()
            
        return Curry(func)
    
    def call(self, *args, **kwargs):
        return self._func(*args, **kwargs)
    
    @property
    def signature(self):
        return self._signature
    
    @property
    def __signature__(self):
        return self._signature
    
    def __repr__(self):
        repr_str = self.signature.__repr__()
        repr_str = repr_str.replace("Signature", f"{self._func.__module__}.{self._func.__name__}", 1)
        return repr_str
    
    def __and__(self, other: Callable):
        if not isinstance(other, Curry):
            other = Curry(other)
        
        def wrapper(*args, **kwargs) -> Any:
            res = self.call(*args, **kwargs)
            if not res:
                return res
            
            return other.call(*args, **kwargs)

        return Curry(wrapper)
    
    def __or__(self, other: Callable):
        if not isinstance(other, Curry):
            other = Curry(other)
                
        def wrapper(*args, **kwargs) -> Any:
            res = self.call(*args, **kwargs)
            if res:
                return res
            
            return other.call(*args, **kwargs)
        
        return Curry(wrapper)
    
    def __invert__(self):
        assert invertible(self.signature.return_annotation), "Return type is not invertible"
        
        def wrapper(*args, **kwargs) -> self.signature.return_annotation:
            return not self.call(*args, **kwargs)
        
        return Curry(wrapper)
    
    def __rshift__(self, other: Callable):
        if not isinstance(other, Curry):
            other = Curry(other)

        def wrapper(*args, **kwargs):
            return other(self(*args, **kwargs))
        
        signature = self.signature.replace(return_annotation=other.signature.return_annotation)
        setattr(wrapper, "__signature__", signature)
        
        return Curry(wrapper)
    
    def __lshift__(self, other: Callable):
        if not isinstance(other, Curry):
            other = Curry(other)

        return other >> self
            
    

def curry(func: Callable):
    return Curry(func)


@curry
def eq(left: Any, right: Any) -> bool:
    return op.eq(left, right)


@curry
def gt(left: Any, right: Any) -> bool:
    return op.gt(left, right)


@curry
def lt(left: Any, right: Any) -> bool:
    return op.lt(left, right)


@curry
def ge(left: Any, right: Any) -> bool:
    return op.ge(left, right)


@curry
def le(left: Any, right: Any) -> bool:
    return op.le(left, right)


@curry
def is_instance(bases: Union[Type, Tuple[Type, ...]], obj: Any) -> bool:
    return isinstance(obj, bases)


@curry
def is_subclass(bases: Union[Type, Tuple[Type, ...]], cls: Type) -> bool:
    return issubclass(cls, bases)


@curry
def is_in(
    collection: Union[List[Any], Dict[Any, Any], Tuple[Any, ...]], val: Any
) -> bool:
    return val in collection


@curry
def peek_nth(index: int, seq: List[Any], default: Optional[Any] = Ellipsis) -> Any:
    if hasattr(seq, "__len__"):
        if 0 <= index < len(seq):
            return seq[index]
        return default

    for i, val in enumerate(seq):
        if i == index:
            return val
        
    if default == Ellipsis:
        raise IndexError("Index out of bound")

    return default


def all(*predicates: List[Callable[[Any], bool]]) -> Callable[[Any], bool]:
    def func(*args, **kwargs):
        for predicate in predicates:
            if not predicate(*args, **kwargs):
                return False
        return True

    return func


def any(*predicates: List[Callable[[Any], bool]]) -> Callable[[Any], bool]:
    def func(*args, **kwargs):
        for predicate in predicates:
            if predicate(*args, **kwargs):
                return True
        return False

    return func


@curry
def sort(iterable: Iterable[Any], key: Callable[[Any], Any]=None):
    return sorted(iterable, key=key)

In [7]:
comparator = gt(right=10) & print

In [9]:
comparator(9)

False

In [16]:
find = any(
    gt(right=10) | identity,
    lt(right=5) | identity
)

In [17]:
find(11)

True

In [ ]:
int.__call__

In [ ]:
is_prime = lambda val: val == 1 or all(val % num for num in range(2, val))

In [ ]:
is_prime(17)

In [ ]:
print_pyramid = lambda val: print("\n".join(
    val[sum(range(i)):sum(range(i)) + i] 
    for i in range(1, len(val))
    if sum(range(i)) < len(val)
))

In [ ]:
print_pyramid("abcdefghjk")

In [ ]:
x = lambda x: x

In [ ]:
x.__call__

In [ ]:
a = 1


In [ ]:
a.__call__

In [ ]:
float.__class__.__module__ == 'builtins'

In [ ]:
def foo(age: int):
    pass

In [ ]:
inspect.signature(foo)